# Training an MLP for FashionMNIST

We revisit FashionMNIST to see whether regularization improves our results.

In [ ]:
from time import perf_counter
from typing import Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torchvision
import sklearn.model_selection

torch.manual_seed(0)


In [ ]:
# load data
train_set_full = torchvision.datasets.FashionMNIST(
    "./data", train=True, download=True, transform=torchvision.transforms.ToTensor()
)
test_set = torchvision.datasets.FashionMNIST(
    "./data", train=False, download=True, transform=torchvision.transforms.ToTensor()
)


In [ ]:
import matplotlib.pyplot as plt

img, target = train_set_full[231]
plt.imshow(img.view(28, 28), cmap="binary")

In [ ]:
val_size = 0.2
train_indices, val_indices = sklearn.model_selection.train_test_split(
    range(len(train_set_full)),
    stratify=train_set_full.targets,
    test_size=val_size,
    random_state=0,
)
train_set = torch.utils.data.Subset(train_set_full, train_indices)
val_set = torch.utils.data.Subset(train_set_full, val_indices)


train_loader = torch.utils.data.DataLoader(train_set, batch_size=500, shuffle=True, num_workers=8)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=1000, shuffle=False, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=500, shuffle=False)


Now define a feedforward neural network that can process 28 by 28 images (e.g., by flattening the image into a single vector).

You can look up network components in the pytorch documentation.

In [ ]:
# Define a model
class FeedForwardNet(nn.Module):
    def __init__(
        self,
        hidden_dim,
        out_dim=10,
        img_shape=(28, 28),
        n_layers: int = 3,
        p: float = 0.5,
    ) -> None:
        super().__init__()
        in_dim = img_shape[0] * img_shape[1]
        self.img_shape = img_shape
        self.layers = nn.ModuleList()

        self.layers.append(nn.Sequential(nn.Linear(in_dim, hidden_dim), nn.ReLU()))
        for _ in range(n_layers - 2):
            self.layers.append(
                nn.Sequential(
                    nn.Linear(hidden_dim, hidden_dim), nn.ReLU(), nn.Dropout(p=p)
                )
            )
        self.layers.append(nn.Linear(hidden_dim, out_dim))

    def forward(self, x):
        """x has shape (batch_size, *img_size)"""
        x = torch.flatten(x, start_dim=1)
        for layer in self.layers:
            x = layer(x)
        return x


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

Add the functionality to use L1 loss in `train_epoch`.

In [9]:
def train_epoch(
    model: torch.nn.Module,
    loader: torch.utils.data.DataLoader,
    criterion: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    l1_coeff:float=0
) -> float:
    """Train a model for one epoch

    Args:
        model (torch.nn.Module): model to be trained
        loader (torch.utils.data.DataLoader): Dataloader for training data
        criterion (torch.nn.Module): loss function
        optimizer (torch.optim.Optimizer): optimizer
        l1_coeff (float): coefficient of L1 loss

    Returns:
        float: total loss over one epoch
    """
    total_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)  # I am using device as a global variable, but you could pass it as well
        out = model(x)
        if l1_coeff != 0:
            params = torch.concat([params.view(-1) for params in model.parameters()])
            l1_loss = F.l1_loss(params, torch.zeros_like(params))
            loss = criterion(out, y) + l1_coeff * l1_loss
        else:
            loss = criterion(out, y)
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return total_loss


@torch.no_grad()  # we dont want these operations to be recorded for automatic differentation, saves memory
def validate(
    model: torch.nn.Module,
    loader: torch.utils.data.DataLoader,
    criterion: torch.nn.Module = None,
) -> Tuple[float, float]:
    """Compute total loss and accuracy

    Args:
        model (torch.nn.Module): model to be evaluated
        loader (torch.utils.data.DataLoader): Dataloader for evaluation data
        criterion (torch.nn.Module, optional): loss function. Defaults to None.

    Returns:
        Tuple[float, float]: total loss, accuracy
    """
    total_loss = 0
    total_correct = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        if criterion is not None:
            loss = criterion(out, y)
            total_loss += loss.item()
        total_correct += (out.argmax(dim=1) == y).sum().item()
    return total_loss, total_correct / len(loader.dataset)



Experiment with a fixed architecture and add regularization one by one. Keep track of what regularization are added. How does your performance change and what is your optimal setup?

In [8]:
hidden_dim = 200
learning_rate = 1e-1

model = FeedForwardNet(hidden_dim=hidden_dim, p=0.5, n_layers=3).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-3, momentum=0.9, nesterov=True)
criterion = nn.CrossEntropyLoss()

n_epochs = 50  # change this as needed
start = perf_counter()
for epoch in range(n_epochs):
    train_epoch(model, train_loader, criterion, optimizer, l1_coeff=0)
    train_loss, train_acc = validate(model, train_loader, criterion=criterion)
    val_loss, val_acc = validate(model, val_loader, criterion=criterion)
    print(
        f"{perf_counter() - start:.1f}s {epoch=}: {train_loss=:.3f}, {train_acc=:.3f}, {val_loss=:.3f}, {val_acc=:.3f}"
    )


In [10]:
test_loss, test_acc = validate(model, test_loader, criterion=criterion)
test_loss, test_acc

(7.048309087753296, 0.8738)

I tried out several regularization combinations, but with my specific setup I did not see large gains in validation or test accuracy.
For example, increasing L2 regularization from 1e-5 to 1e-3 does decrease the gap between training and validation accuracy, but it does not increase test performance.
One reason may be that the fully connected network is not particularly suited for this problem.

| Layers | Hidden | LR   | Epochs | Optimizer                   | Batch size | Dropout | L1   | L2   | Early Stopping | Train Acc | Val Acc | Test Acc |
|--------|--------|------|--------|-----------------------------|------------|---------|------|------|----------------|-----------|---------|----------|
| 3      | 200    | 1e-1 | 50     | SGD (Nesterov momentum=0.9) | 500        | 0       | 0    | 0    | No             | 0.971     | 0.888   | 0.8815   |
| 3      | 200    | 1e-1 | 50     | SGD (Nesterov momentum=0.9) | 500        | 0.5     | 0    | 0    | No             | 0.953     | 0.891   |          |
| 3      | 200    | 1e-1 | 50     | SGD (Nesterov momentum=0.9) | 500        | 0.5     | 0    | 1e-5 | No             | 0.953     | 0.889   |  0.8825  |
| 3      | 200    | 1e-1 | 50     | SGD (Nesterov momentum=0.9) | 500        | 0.5     | 0    | 1e-3 | No             | 0.901     | 0.874   |  0.8757  |
| 3      | 200    | 1e-1 | 50     | SGD (Nesterov momentum=0.9) | 500        | 0.5     | 1e-5 | 1e-5 | No             | 0.955     | 0.889   | 0.8871   |